In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, RobertaForSequenceClassification
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch.utils.data as data_utils
import torch


device = torch.device("mps" if torch.mps.is_available() else 
                      "cuda" if torch.cuda.is_available() else "cpu")
device

/Users/lucabadiali/Desktop/professionAI/modulo9/Project/MLOPS_Project/ProjectEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='mps')

In [2]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [3]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
labels

['negative', 'neutral', 'positive']

In [7]:

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
tokenizer = AutoTokenizer.from_pretrained(MODEL)
text = "today I ate some pasta"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.6674
2) positive 0.3132
3) negative 0.0194


### Tentativo di training

In [21]:
import numpy as np
import evaluate

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, concatenate_datasets

def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=128, truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')


dataset = load_dataset('tweet_eval', 'sentiment')




In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # avoid fork/parallelism warnings on macOS

import torch
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    DataCollatorWithPadding
)
import evaluate

# --- Device detection ---
if torch.cuda.is_available():
    device = "cuda"
    use_bf16 = torch.cuda.is_bf16_supported()
    use_fp16 = not use_bf16
elif torch.backends.mps.is_available():
    device = "mps"
    use_bf16 = False
    use_fp16 = False
else:
    device = "cpu"
    use_bf16 = False
    use_fp16 = False

MODEL_NAME = "cardiffnlp/twitter-roberta-base-sep2022"

# --- Tokenizer: keep short max_length to save memory ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, model_max_length=128)

def tokenize_function(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=128,
        padding=False  # we will pad per-batch via DataCollatorWithPadding
    )

# If your dataset column is "label", keep it; Trainer can handle it.
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# --- Data collator that pads dynamically ---
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    pad_to_multiple_of=8 if (device == "cuda" and (use_bf16 or use_fp16)) else None
)

# --- Model dtype choice ---
if device == "cuda" and use_bf16:
    load_dtype = torch.bfloat16
elif device == "cuda" and use_fp16:
    load_dtype = torch.float16
else:
    load_dtype = torch.float32  # MPS/CPU -> fp32

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3, torch_dtype=load_dtype
)
model.gradient_checkpointing_enable()
model.config.use_cache = False

# --- Training args: stop forking on macOS, fix pin_memory ---
trainer_fp16 = bool(device == "cuda" and use_fp16)
trainer_bf16 = bool(device == "cuda" and use_bf16)

training_args = TrainingArguments(
    output_dir="artifacts",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",

    eval_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    save_steps=500,

    load_best_model_at_end=True,
    metric_for_best_model="recall",
    greater_is_better=True,
    save_total_limit=2,

    # Precision
    fp16=trainer_fp16,
    bf16=trainer_bf16,

    # DataLoader knobs (avoid fork/tokenizers warning on macOS)
    dataloader_num_workers=0,                         # <- key for macOS/MPS
    dataloader_pin_memory=(device == "cuda"),         # False on MPS/CPU, True on CUDA
    group_by_length=True,
    report_to="none",
)

# --- Metrics (macro recall, etc.) ---
recall_metric = evaluate.load("recall")
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
        "recall": recall_metric.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]

small_train = tokenized_datasets["train"].select(range(100))
small_eval  = tokenized_datasets["validation"].select(range(1))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_ds,
    eval_dataset= eval_ds,
    compute_metrics=compute_metrics,
    data_collator=data_collator,       # <- important
    tokenizer=tokenizer,
    callbacks=callbacks,
)

# Optional explicit device move (Trainer usually handles it)
model.to(device)
trainer.train()

trainer.save_model("saved_model")
tokenizer.save_pretrained("saved_model")
try:
    trainer.create_model_card()
except Exception:
    pass


`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sep2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/nc/1wpyndzx5ps8nbt0b5zm9jx80000gn/T/ipykernel_2067/3094520460.py:119: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,F1 Macro,Recall
500,0.594100,0.668015,0.716000,0.697675,0.702171


In [20]:
text = "The second law of thermodynamics is about entropy"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt').to(device)
output = model(**encoded_input)
scores = output[0][0].detach().cpu().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.7894
2) positive 0.1149
3) negative 0.0957


In [9]:
# ---- COPY-PASTE FROM HERE ----
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from datasets import DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding

def make_trainer_ready(
    raw_ds: DatasetDict,
    model_name: str = "cardiffnlp/twitter-roberta-base-sep2022",
    train_frac: float = 0.2,
    val_frac: float = 0.2,
    seed: int = 42,
    label_col: str = "label",
    text_col: str = "text",
    max_length: int = 128,
    pad_to_multiple_of_8_on_cuda: bool = True,
):
    """
    Returns (train_ds, eval_ds, data_collator, tokenizer) ready for HF Trainer.
    - Ensures there's a validation split (creates one from train if missing).
    - Takes fractional subsets, stratified by label when possible.
    - Tokenizes and keeps only the columns Trainer expects.
    """
    assert 0 < train_frac <= 1.0, "train_frac must be in (0,1]."
    assert 0 < val_frac <= 1.0, "val_frac must be in (0,1]."
    assert text_col in raw_ds["train"].column_names, f"Missing text column: {text_col}"
    assert label_col in raw_ds["train"].column_names, f"Missing label column: {label_col}"

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, model_max_length=max_length)

    # 1) Ensure we have a validation split
    if "validation" not in raw_ds:
        split = raw_ds["train"].train_test_split(
            test_size=val_frac,
            stratify_by_column=label_col if label_col in raw_ds["train"].column_names else None,
            seed=seed,
        )
        raw_ds = DatasetDict(train=split["train"], validation=split["test"])
    else:
        raw_ds = DatasetDict(train=raw_ds["train"], validation=raw_ds["validation"])

    # 2) Take fractions (stratified when possible)
    def take_frac(ds, frac):
        if frac >= 1.0:  # keep full split
            return ds
        out = ds.train_test_split(
            test_size=1 - frac,
            stratify_by_column=label_col if label_col in ds.column_names else None,
            seed=seed,
        )
        return out["train"]  # the kept fraction

    small_train = take_frac(raw_ds["train"], train_frac)
    small_eval  = take_frac(raw_ds["validation"], val_frac)

    # 3) Tokenize (no padding here; we pad per-batch with the collator)
    def tok(batch):
        return tokenizer(batch[text_col], truncation=True, max_length=max_length, padding=False)

    small_train_tok = small_train.map(tok, batched=True, remove_columns=[c for c in small_train.column_names if c not in (text_col, label_col)])
    small_eval_tok  = small_eval.map(tok,  batched=True, remove_columns=[c for c in small_eval.column_names  if c not in (text_col, label_col)])

    # 4) Keep only the columns Trainer needs
    keep_cols = ["input_ids", "attention_mask", label_col]
    small_train_tok = small_train_tok.remove_columns([c for c in small_train_tok.column_names if c not in keep_cols])
    small_eval_tok  = small_eval_tok.remove_columns([c for c in small_eval_tok.column_names  if c not in keep_cols])

    # 5) Data collator with dynamic padding (CUDA gets pad_to_multiple_of=8)
    import torch
    pad_to_mult = 8 if (pad_to_multiple_of_8_on_cuda and torch.cuda.is_available()) else None
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=pad_to_mult)

    return small_train_tok, small_eval_tok, data_collator, tokenizer

# ---- USAGE EXAMPLE ----
# Assumes you already have `dataset` (a DatasetDict with 'train' (and maybe 'validation')).
# Example:
# from datasets import load_dataset
# dataset = load_dataset("tweet_eval", "sentiment")

train_ds, eval_ds, data_collator, tokenizer = make_trainer_ready(
    raw_ds=dataset,
    model_name="cardiffnlp/twitter-roberta-base-sep2022",
    train_frac=0.2,    # take 20% of train
    val_frac=0.5,      # take 50% of validation
    seed=42,
    label_col="label",
    text_col="text",
    max_length=128,
)

# Pass directly to Trainer:
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
# model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sep2022", num_labels=3)
# args = TrainingArguments(output_dir="out", per_device_train_batch_size=4, per_device_eval_batch_size=8, evaluation_strategy="epoch", report_to="none")
# trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, data_collator=data_collator, tokenizer=tokenizer)
# trainer.train()
# ---- COPY-PASTE UNTIL HERE ----


Map: 100%|██████████| 1000/1000 [00:00<00:00, 10983.68 examples/s]


In [11]:
eval_ds

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})